# 필요한 환경 불러오기

## 1. 사전학습 모델을 불러오기 위한 transformers를 설치합니다.

In [ ]:
!pip install transformers

## 2. 필요한 모듈을 불러옵니다.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import time
import os

# 3. 문장 분류 

## 분류기 정의하기
- 문장들들 사이의 관계를 분류할 수 있는 classifier함수를 정의합니다.

In [ ]:
def inference(model, premise, hypothesis):
    tokenizer = BertTokenizer.from_pretrained(model)
    model = BertForSequenceClassification.from_pretrained(model)
    
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        entail_score = round(prob[0][0].item(), 2)
        contra_score = round(prob[0][1].item(), 2)
        neutral_score = round(prob[0][2].item(), 2)
        if torch.argmax(prob) == 0:
            model_pred = "함의(entailment)"
        elif torch.argmax(prob) == 1:
            model_pred = "모순(contradiction)"
        else:
            model_pred = "중립(neutral)"
            
    result_list = [entail_score, contral_score, neutral_score]

    result_series = pd.Series(result_list, index = ['entailment', 'contradiction', 'neutral'])

    model_pred = result_series.idxmax()
     title = "Model Prediction :" + model_pred
    
    plt.pie(result_series, autopct = '%.1f%%', shadow = True,textprops = {'fontsize':15})
    plt.legend(labels=result_series.index, bbox_to_anchor=(0.95, 0.65))
    plt.title(title, size = 20)
    plt.show()

In [ ]:
def inference_pipeline(model, premise, hypothesis):
  classifier = pipeline(
    "text-classification",
    model,
    return_all_scores=True,
  )
  
  classifier_input = premise + '[SEP]' + hypothesis

  result = classifier(classifier_input)

  entail_score = result[0]['score']
  contral_score = result[2]['score']
  neutral_score = result[1]['score']

  result_list = [entail_score, neutral_score, contral_score]

  result_series = pd.Series(result_list, index = ['entailment', 'neutral', 'contradiction'])

  model_pred = result_series.idxmax()
  title = "Model Prediction :" + model_pred

  plt.pie(result_series, autopct = '%.1f%%', shadow = True,textprops = {'fontsize':15})
  plt.legend(labels=result_series.index, bbox_to_anchor=(0.95, 0.65))
  plt.title(title, size = 20)
  plt.show()

# 4. 결과 확인하기

In [ ]:
model = ""

premise = input("전제 문장을 입력해 주세요: ")
hypothesis = input("가설 문장을 입력해 주세요: ")

inference_pipeline(target_label, premise, hypothesis)